<a href="https://colab.research.google.com/github/mangrue/IDD-Resources/blob/main/IDD_CrossModelsDatasets_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Setting up environment**

In [ ]:
#Install specific libraries
#! pip install torch
#! pip install transformers
#! pip install pycaret
#! pip install pandas
#! pip install numpy
#! pip install pycaret
#! pip install matplotlib
#! pip install -U scikit-learn
#! pip install transformers==2.8.0
#!pip install --upgrade huggingface_hub
!pip install evaluate -q
!pip install datasets -q
!pip install -U datasets

import numpy as np
import pandas as pd
#import pycaret
import transformers
from transformers import AutoModel, BertTokenizerFast
from transformers import BertTokenizer, BertForMaskedLM
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
# specify GPU
device = torch.device("cuda")

# Set Working Directory - if working on Google Drive
# Mount Google Drive - applicable, if working on Google Drive
from google.colab import drive
drive.mount('/content/drive')
#%cd /content/drive/MyDrive #/Colab_Notebooks


#**Hugging Face**

In [ ]:
#login to hugging face
!huggingface-cli login --token #hugging face token #--add-to-git-credential

from transformers import pipeline

#**Load dataset(s)**

In [ ]:
#Check dataset(s)

#!pip install -U datasets
from datasets import load_dataset

#Load and check data
dataset = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/LIAR_final.csv", token=True)
#dataset = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/ISOT_final.csv", token=True)   #use_auth_token deprecated since v5 of Transformers, Sept. 2024 (use "token" instead)
#dataset = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/BuzzFeed-2016_final.csv", token=True)
#dataset = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/NELA-GT-2018_final.csv", token=True)
#app. 20% of NELA-GT-2018 dataset loaded
#dataset = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/NELA-GT-2022_final.csv", token=True)
#app. 10% of NELA-GT-2022 dataset loaded

#dataset = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/WELFake_final.csv", token=True)

print('Dataset info:')
print('Size: ' + str(len(dataset)))

dataset

dataset.set_format(type='pandas')
df = dataset['train'][:]
print(df)

df.sample(n=10)

#Check if None values exist in rows and then check for duplicates
#null_mask = df.isnull().any(axis=1)
#null_rows = df[null_mask]
#print(f"Null rows: {null_rows}")

#item0 = df.shape[0]
#df = df.drop_duplicates()
#item1 = df.shape[0]
#print(f"Duplicates found: {item0-item1}")

**Load dataset(s) and libraries**

In [ ]:
from datasets import load_dataset
from evaluate import evaluator
import evaluate
from transformers import AutoModelForSequenceClassification, pipeline
from datasets import Dataset
from transformers import AutoTokenizer

eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/LIAR_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))                                    #LIAR
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/ISOT_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))                                   #ISOT (title+text)
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/ISOT_final_only_title.csv", token=True, split="train").shuffle(seed=42).select(range(500))                        #ISOT (only title)
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/ISOT_final_only_text.csv", token=True, split="train").shuffle(seed=42).select(range(500))                         #ISOT (only text, title not merged)
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/BuzzFeed-2016_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))                          #BuzzFeed (merged title + text, like all other datasets)
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/BuzzFeed-2016_final_titles_sentence_case.csv", token=True, split="train").shuffle(seed=42).select(range(500))     #BuzzFeed (final sentence cased)   #FINAL ONE USED FOR EVALUATION
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/BuzzFeed-2016_final_only_text.csv", token=True, split="train").shuffle(seed=42).select(range(500))                #BuzzFeed (only text)
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS//NELA-GT-2018_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))                          #NELA-GT-2018
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS//NELA-GT-2022_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))                          #NELA-GT-2022

#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/WELFake_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))                    #WELFake
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/clmentbisaillon_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))            #ISOT
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", token=True, split="train").shuffle(seed=42).select(range(500)) #EU news / EUvsDisinfo
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/COVID19-fake-news_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))          #COVID-19 content

#EU mixed data
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_1_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_2_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_3_EMNAD_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_4_enr_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_5_enr_Belgium_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_6_enr_Belgium_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))
#Each MS/country
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_Belgium_EUvsDis_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_Cyprus_EUvsDis_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_Ireland_EUvsDis_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_Malta_EUvsDis_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))
#eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_United_Kingdom_EUvsDis_final.csv", token=True, split="train").shuffle(seed=42).select(range(500))

#print(eval_data)

#df = pd.DataFrame(eval_data)
#df = df[["text", "label"]]

#df = df.sample(frac=1, random_state=42).reset_index(drop=True).head(100)  #limit by head or sample size of data sample
#print(df)
#display(df)

#**Evaluation of model's performance on certain dataset(s)**

**1) MODEL: dima806/fake-news-classifier (pre-trained on: WELFake)**

In [ ]:
#example

MODEL = "dima806/fake-news-classifier"
clf = pipeline("text-classification", model=MODEL, tokenizer=MODEL)

#Small samples' classification

# testing on unseen data #example from below from LIAR trained model example
#unseen_news_text = ["Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing",     # Fake
#                    "WATCH: George W. Bush Calls Out Trump For Supporting White Supremacy",               # Fake
#                    "U.S. lawmakers question businessman at 2016 Trump Tower meeting: sources",           # True
#                    "Trump administration issues new rules on U.S. visa waivers"                          # True
#                    ]

# random examples from LIAR dataset

df = pd.DataFrame(eval_data)
sampledf = df.sample(n=10)
sample = sampledf[["text", "label"]]
print(sample)
data = sampledf["text"]
data = data.values.tolist()
print(data)

i = 0
while i < len(data):
    text = data[i]
    result = clf(text[:512])
    print(result)
    i += 1

# make a classification pipeline
#sample_title = '''Elon Musk buys Twitter'''
#pipe = pipeline("text-classification", model=MODEL, tokenizer=MODEL)
#sample_title = '''Elon Musk buys Twitter'''
#pipe(sample_title, top_k=None)
# output of trained model matches claimed output for this example

#from transformers import pipeline
#MODEL = "jy46604790/Fake-News-Bert-Detect"
#clf = pipeline("text-classification", model=MODEL, tokenizer=MODEL)

#text = "Indonesian police have recaptured a U.S. citizen who escaped a week ago from an overcrowded prison on the holiday island of Bali, the jail s second breakout of foreign inmates this year.  Cristian Beasley from California was rearrested on Sunday, Badung Police chief Yudith Satria Hananta said, without providing further details.  Beasley was a suspect in crimes related to narcotics but had not been sentenced when he escaped from Kerobokan prison in Bali last week. The 32-year-old is believed to have cut through bars in the ceiling of his cell before scaling a perimeter wall of the prison in an area being refurbished. The Kerobokan prison, about 10 km (six miles) from the main tourist beaches in the Kuta area, often holds foreigners facing drug-related charges. Representatives of Beasley could not immediately be reached for comment. In June, an Australian, a Bulgarian, an Indian and a Malaysian tunneled to freedom about 12 meters (13 yards) under Kerobokan prison s walls. The Indian and the Bulgarian were caught soon after in neighboring East Timor, but Australian Shaun Edward Davidson and Malaysian Tee Kok King remain at large. Davidson has taunted authorities by saying he was enjoying life in various parts of the world, in purported posts on Facebook.  Kerobokan has housed a number of well-known foreign drug convicts, including Australian Schappelle Corby, whose 12-1/2-year sentence for marijuana smuggling got huge media attention."

#result = clf(text)
#result

In [ ]:
#EVALUATE model's performance on the data

MODEL = "dima806/fake-news-classifier"
task_evaluator = evaluator("text-classification")
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# 1. Pass a model name or path
eval_results = task_evaluator.compute(
    model_or_pipeline=MODEL,
    tokenizer=AutoTokenizer.from_pretrained(MODEL, max_len=512),
    data=eval_data,
    label_mapping={"FAKE": 0, "REAL": 1}
)

# 2. Pass an instantiated model
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=AutoTokenizer.from_pretrained(MODEL, max_len=512),
    data=eval_data,
    label_mapping={"FAKE": 0, "REAL": 1}
)

# 3. Pass an instantiated pipeline
pipe = pipeline("text-classification", model=MODEL, tokenizer=AutoTokenizer.from_pretrained(MODEL, max_len=512))

eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=eval_data,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={"FAKE": 0, "REAL": 1},
    strategy="bootstrap", #calculate confidence intervals using the boostrap method
    n_resamples=200 #use 200 for n_resamples (default is 9999), default for confidence_level is 0.95
)
print("___________")
print("dima806")
print("EVALUATION RESULTS:")
print("___________")
print(eval_results)

**2) MODEL: hamzab/roberta-fake-news-classification (pre-trained on: ISOT)**





In [ ]:
#FORMAT input: <title> TITLE HERE <content> CONTENT HERE <end>

MODEL = AutoModelForSequenceClassification.from_pretrained("hamzab/roberta-fake-news-classification")
task_evaluator = evaluator("text-classification")

tokenizer = AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification") #, config=AutoConfig.from_pretrained("hamzab/roberta-fake-news-classification"))

#tokenizer = AutoTokenizer.from_pretrained(MODEL)

#import torch
#def predict_fake(title,text):
#    input_str = "<title>" + title + "<content>" +  text + "<end>"
#    input_ids = tokenizer.encode_plus(input_str, max_length=512, padding="max_length", truncation=True, return_tensors="pt")
#    device =  'cuda' if torch.cuda.is_available() else 'cpu'
#    model.to(device)
#    with torch.no_grad():
#        output = model(input_ids["input_ids"].to(device), attention_mask=input_ids["attention_mask"].to(device))
#    return dict(zip(["Fake","Real"], [x.item() for x in list(torch.nn.Softmax()(output.logits)[0])] ))#

#print(predict_fake(<HEADLINE-HERE>,<CONTENT-HERE>))

#import gradio as gr
#iface = gr.Interface(fn=predict_fake, inputs=[gr.inputs.Textbox(lines=1,label="headline"),gr.inputs.Textbox(lines=6,label="content")], outputs="label").launch(share=True)

# 1. Pass a model name or path
eval_results = task_evaluator.compute(
    model_or_pipeline=MODEL,
    tokenizer=AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification"),
    data=eval_data,
    label_mapping={"FAKE": 0, "TRUE": 1}
)

# 2. Pass an instantiated model
model = AutoModelForSequenceClassification.from_pretrained("hamzab/roberta-fake-news-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification"),
    data=eval_data,
    label_mapping={"FAKE": 0, "TRUE": 1}
)

# 3. Pass an instantiated pipeline
pipe = pipeline("text-classification", model=MODEL, tokenizer=AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification"))

eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=eval_data,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={"FAKE": 0, "TRUE": 1},
    strategy="bootstrap",
    n_resamples=200
)
print("___________")
print("hamzab")
print("EVALUATION RESULTS:")
print("___________")
print(eval_results)

**-) Special evaluation regarding text with and without title**

In [ ]:
from datasets import load_dataset
from evaluate import evaluator
import evaluate
from transformers import AutoModelForSequenceClassification, pipeline
from datasets import Dataset
from transformers import AutoTokenizer

seed_count = 42

while seed_count < 52:

  #eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/BuzzFeed-2016_final_only_text.csv", use_auth_token=True, split="train").shuffle(seed=seed_count).select(range(500))     #BuzzFeed (only text)
  #eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/BuzzFeed-2016_final.csv", use_auth_token=True, split="train").shuffle(seed=seed_count).select(range(500))     #BuzzFeed (merged title + text, like all other datasets)
  eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/BuzzFeed-2016_final_titles_sentence_case.csv", token=True, split="train").shuffle(seed=seed_count).select(range(500))     #BuzzFeed (final sentence cased)

  #Store samples for closer inspection
  #df = pd.DataFrame(eval_data)
  #df = df[["text", "label"]]
  #df.to_csv("/content/drive/MyDrive/Colab_Notebooks/data_samples_hamzab_on_BuzzFeed_special_evaluation/sample_seed_" + str(seed_count) + ".csv", sep=',', index=False, encoding='utf-8')

  #FORMAT input: <title> TITLE HERE <content> CONTENT HERE <end>

  MODEL = AutoModelForSequenceClassification.from_pretrained("hamzab/roberta-fake-news-classification")
  task_evaluator = evaluator("text-classification")

  tokenizer = AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification") #, config=AutoConfig.from_pretrained("hamzab/roberta-fake-news-classification"))

  # 1. Pass a model name or path
  eval_results = task_evaluator.compute(
    model_or_pipeline=MODEL,
    tokenizer=AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification"),
    data=eval_data,
    label_mapping={"FAKE": 0, "TRUE": 1}
  )

  # 2. Pass an instantiated model
  model = AutoModelForSequenceClassification.from_pretrained("hamzab/roberta-fake-news-classification")

  eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification"),
    data=eval_data,
    label_mapping={"FAKE": 0, "TRUE": 1}
  )

  # 3. Pass an instantiated pipeline
  pipe = pipeline("text-classification", model=MODEL, tokenizer=AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification"))

  eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=eval_data,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={"FAKE": 0, "TRUE": 1},
    strategy="bootstrap",
    n_resamples=200
  )
  print("___________")
  print("hamzab")
  print("EVALUATION RESULTS (SEED:" + str(seed_count) + "):")
  print("___________")
  print(eval_results)
  seed_count += 1

**3) MODEL: jy46604790/Fake-News-Bert-Detect (pre-trained on ISOT)**

In [ ]:
MODEL = "jy46604790/Fake-News-Bert-Detect"
#clf = pipeline("text-classification", model=MODEL, tokenizer=MODEL)

#text = "Indonesian police have recaptured a U.S. citizen who escaped a week ago from an overcrowded prison on the holiday island of Bali, the jail s second breakout of foreign inmates this year.  Cristian Beasley from California was rearrested on Sunday, Badung Police chief Yudith Satria Hananta said, without providing further details.  Beasley was a suspect in crimes related to narcotics but had not been sentenced when he escaped from Kerobokan prison in Bali last week. The 32-year-old is believed to have cut through bars in the ceiling of his cell before scaling a perimeter wall of the prison in an area being refurbished. The Kerobokan prison, about 10 km (six miles) from the main tourist beaches in the Kuta area, often holds foreigners facing drug-related charges. Representatives of Beasley could not immediately be reached for comment. In June, an Australian, a Bulgarian, an Indian and a Malaysian tunneled to freedom about 12 meters (13 yards) under Kerobokan prison s walls. The Indian and the Bulgarian were caught soon after in neighboring East Timor, but Australian Shaun Edward Davidson and Malaysian Tee Kok King remain at large. Davidson has taunted authorities by saying he was enjoying life in various parts of the world, in purported posts on Facebook.  Kerobokan has housed a number of well-known foreign drug convicts, including Australian Schappelle Corby, whose 12-1/2-year sentence for marijuana smuggling got huge media attention."

#result = clf(text)
#result

task_evaluator = evaluator("text-classification")

tokenizer = AutoTokenizer.from_pretrained("jy46604790/Fake-News-Bert-Detect")

# 1. Pass a model name or path
eval_results = task_evaluator.compute(
    model_or_pipeline=MODEL,
    tokenizer=AutoTokenizer.from_pretrained("jy46604790/Fake-News-Bert-Detect"),
    data=eval_data,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1} #LABEL_0: Fake news // LABEL_1: Real news
)

# 2. Pass an instantiated model
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=AutoTokenizer.from_pretrained("jy46604790/Fake-News-Bert-Detect"),
    data=eval_data,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1} #LABEL_0: Fake news // LABEL_1: Real news
)

# 3. Pass an instantiated pipeline
pipe = pipeline("text-classification", model=MODEL, tokenizer=AutoTokenizer.from_pretrained("jy46604790/Fake-News-Bert-Detect"))

eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=eval_data,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={"LABEL_0": 0, "LABEL_1": 1}, #LABEL_0: Fake news // LABEL_1: Real news
    strategy="bootstrap",
    n_resamples=200
)
print("___________")
print("jy46604790")
print("EVALUATION RESULTS:")
print("___________")
print(eval_results)

**4) MODEL: winterForestStump/Roberta-fake-news-detector (pre-trained on ISOT and EU news)**

In [ ]:
MODEL = "winterForestStump/Roberta-fake-news-detector"

#clf = pipeline("text-classification", model=MODEL, tokenizer=MODEL)

#text = "From the very beginning, the EU has been extremely non-transparent. The deployment of the European Union presence in Armenia was carried out forcefully, under serious pressure from Brussels"

#result = clf(text)
#result

task_evaluator = evaluator("text-classification")

tokenizer = AutoTokenizer.from_pretrained("winterForestStump/Roberta-fake-news-detector")

# 1. Pass a model name or path
eval_results = task_evaluator.compute(
    model_or_pipeline=MODEL,
    tokenizer=AutoTokenizer.from_pretrained("winterForestStump/Roberta-fake-news-detector"),
    data=eval_data,
    label_mapping={"FAKE": 0, "TRUE": 1}
)

# 2. Pass an instantiated model
model = AutoModelForSequenceClassification.from_pretrained("winterForestStump/Roberta-fake-news-detector")

eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=AutoTokenizer.from_pretrained("winterForestStump/Roberta-fake-news-detector"),
    data=eval_data,
    label_mapping={"FAKE": 0, "TRUE": 1}
)

# 3. Pass an instantiated pipeline
pipe = pipeline("text-classification", model=MODEL, tokenizer=AutoTokenizer.from_pretrained("winterForestStump/Roberta-fake-news-detector"))

eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=eval_data,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={"FAKE": 0, "TRUE": 1},
    strategy="bootstrap",
    n_resamples=200
)
print("___________")
print("winterForestStump")
print("EVALUATION RESULTS:")
print("___________")
print(eval_results)

**-) Run EU model over files with different seeds (42, 48, and 54)**

In [ ]:
import os
from datasets import load_dataset
from evaluate import evaluator
import evaluate
from transformers import AutoModelForSequenceClassification, pipeline
from datasets import Dataset
from transformers import AutoTokenizer

seed_count = 42
files_dir = "/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS"
eval_data = ""
file_name = ""

while seed_count <= 54:

  for file_name in sorted(os.listdir(files_dir)):
    f = os.path.join(files_dir, file_name)
    if os.path.isfile(f):
      #print(file_name)
      eval_data = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/"+file_name, token=True, split="train").shuffle(seed=seed_count).select(range(500))

      MODEL = "winterForestStump/Roberta-fake-news-detector"
      task_evaluator = evaluator("text-classification")
      tokenizer = AutoTokenizer.from_pretrained("winterForestStump/Roberta-fake-news-detector")

      # 1. Pass a model name or path
      eval_results = task_evaluator.compute(
        model_or_pipeline=MODEL,
        tokenizer=AutoTokenizer.from_pretrained("winterForestStump/Roberta-fake-news-detector"),
        data=eval_data,
        label_mapping={"FAKE": 0, "TRUE": 1}
      )

      # 2. Pass an instantiated model
      model = AutoModelForSequenceClassification.from_pretrained("winterForestStump/Roberta-fake-news-detector")

      eval_results = task_evaluator.compute(
        model_or_pipeline=model,
        tokenizer=AutoTokenizer.from_pretrained("winterForestStump/Roberta-fake-news-detector"),
        data=eval_data,
        label_mapping={"FAKE": 0, "TRUE": 1}
      )

      # 3. Pass an instantiated pipeline
      pipe = pipeline("text-classification", model=MODEL, tokenizer=AutoTokenizer.from_pretrained("winterForestStump/Roberta-fake-news-detector"))

      eval_results = task_evaluator.compute(
        model_or_pipeline=pipe,
        data=eval_data,
        metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
        label_mapping={"FAKE": 0, "TRUE": 1},
        strategy="bootstrap",
        n_resamples=200
      )
      print("___________")
      print("winterForestStump")
      print("EVALUATION RESULTS (" + "FILE: " + file_name + "; SEED:" + str(seed_count) + "):")
      print("___________")
      print(eval_results)
  seed_count += 6

**5) MODEL: XSY/albert-base-v2-fakenews-discriminator (pre-trained on: ISOT)**

In [ ]:
MODEL = "XSY/albert-base-v2-fakenews-discriminator"

task_evaluator = evaluator("text-classification")

tokenizer = AutoTokenizer.from_pretrained("XSY/albert-base-v2-fakenews-discriminator")

# 1. Pass a model name or path
eval_results = task_evaluator.compute(
    model_or_pipeline=MODEL,
    tokenizer=AutoTokenizer.from_pretrained("XSY/albert-base-v2-fakenews-discriminator"),
    data=eval_data,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1} #label_0 : Fake news // label_1 : Real news
)

# 2. Pass an instantiated model
model = AutoModelForSequenceClassification.from_pretrained("XSY/albert-base-v2-fakenews-discriminator")

eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=AutoTokenizer.from_pretrained("XSY/albert-base-v2-fakenews-discriminator"),
    data=eval_data,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1} #label_0 : Fake news // label_1 : Real news
)

# 3. Pass an instantiated pipeline
pipe = pipeline("text-classification", model=MODEL, tokenizer=AutoTokenizer.from_pretrained("XSY/albert-base-v2-fakenews-discriminator"))

eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=eval_data,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={"LABEL_0": 0, "LABEL_1": 1}, #label_0 : Fake news // label_1 : Real news
    strategy="bootstrap",
    n_resamples=200
)
print("___________")
print("XSY")
print("EVALUATION RESULTS:")
print("___________")
print(eval_results)

**6) MODEL: Denyol/FakeNews-deberta-base (pre-trained on: COVID-19 fake news)**

In [ ]:
MODEL = "Denyol/FakeNews-deberta-base"

task_evaluator = evaluator("text-classification")

tokenizer = AutoTokenizer.from_pretrained("Denyol/FakeNews-deberta-base")

# 1. Pass a model name or path
eval_results = task_evaluator.compute(
    model_or_pipeline=MODEL,
    tokenizer=AutoTokenizer.from_pretrained("Denyol/FakeNews-deberta-base"),
    data=eval_data,
    label_mapping={"FAKE": 0, "REAL": 1} #0: FAKE, 1: TRUE
)

# 2. Pass an instantiated model
model = AutoModelForSequenceClassification.from_pretrained("Denyol/FakeNews-deberta-base")

eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=AutoTokenizer.from_pretrained("Denyol/FakeNews-deberta-base"),
    data=eval_data,
    label_mapping={"FAKE": 0, "REAL": 1} #0: FAKE, 1: TRUE
)

# 3. Pass an instantiated pipeline
pipe = pipeline("text-classification", model=MODEL, tokenizer=AutoTokenizer.from_pretrained("Denyol/FakeNews-deberta-base"))

eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=eval_data,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={"FAKE": 0, "REAL": 1}, #0: FAKE, 1: TRUE
    strategy="bootstrap",
    n_resamples=200
)
print("___________")
print("Denyol")
print("EVALUATION RESULTS:")
print("___________")
print(eval_results)

**7) MODEL: AlexanderHolmes0/fake-news-detector-long (pre-trained on: ISOT)**

In [ ]:
from datasets import load_dataset
#from evaluate import evaluator
#import evaluate
#from transformers import AutoModelForSequenceClassification, pipeline
from datasets import Dataset
#from transformers import AutoTokenizer

print("Before label change:", eval_data)
df = pd.DataFrame(eval_data)
df = df[["text", "label"]]
df = df.sample(frac=1, random_state=42).reset_index(drop=True).head(100)  #limit by head or sample size of data sample
print(df)

#only for this model (1=FALSE, 0=TRUE) - change label accordingly before evaluation (usually boolean true = binary 1 and boolean false = binary 0)
def map_labels(eval_data):
    eval_data["label"] = not eval_data["label"]
    return eval_data

result = eval_data.map(map_labels)

print("----------------------------")
print("After label change:", result)
df = pd.DataFrame(result)
df = df[["text", "label"]]
df = df.sample(frac=1, random_state=42).reset_index(drop=True).head(100)  #limit by head or sample size of data sample
print(df)

eval_data = result

In [ ]:
MODEL = "AlexanderHolmes0/fake-news-detector-long"

task_evaluator = evaluator("text-classification")

tokenizer = AutoTokenizer.from_pretrained("AlexanderHolmes0/fake-news-detector-long")

# 1. Pass a model name or path
eval_results = task_evaluator.compute(
    model_or_pipeline=MODEL,
    tokenizer=AutoTokenizer.from_pretrained("AlexanderHolmes0/fake-news-detector-long"),
    data=eval_data,
    #label_mapping={"true": 1, "fake": 0} #1: true, 0: fake in evaluation data
    label_mapping={"true": 0, "fake": 1}
)

# 2. Pass an instantiated model
model = AutoModelForSequenceClassification.from_pretrained("AlexanderHolmes0/fake-news-detector-long")

eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=AutoTokenizer.from_pretrained("AlexanderHolmes0/fake-news-detector-long"),
    data=eval_data,
    #label_mapping={"true": 1, "fake": 0} #1: true, 0: fake in evaluation data
    label_mapping={"true": 0, "fake": 1}
)

# 3. Pass an instantiated pipeline
pipe = pipeline("text-classification", model=MODEL, tokenizer=AutoTokenizer.from_pretrained("AlexanderHolmes0/fake-news-detector-long"))

eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=eval_data,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    #label_mapping={"true": 1, "fake": 0} #1: true, 0: fake in evaluation data
    label_mapping={"true": 0, "fake": 1},
    strategy="bootstrap",
    n_resamples=200
)
print("___________")
print("AlexanderHolmes0")
print("EVALUATION RESULTS:")
print("___________")
print(eval_results)